# 04 — Obligation Advisor Agent

Given a role (provider/deployer/importer) and risk level, produces a compliance checklist with specific obligations from the EU AI Act. Supports multi-turn conversation and can chain from the Compliance Classifier's output.

In [ ]:
import sys
sys.path.insert(0, "..")

from src.tools import init_tools, TOOL_SCHEMAS
from src.agent import run_agent
from src.models import ObligationChecklist
from src.embeddings import get_openai_client

LANG = "de"  # change to "en" for English

client = get_openai_client()
init_tools(chroma_path="../chroma_db", lang=LANG)

In [ ]:
ADVISOR_SYSTEM_PROMPT = """Du bist ein Pflichten-Berater für die EU-KI-Verordnung (AI Act). Gegeben eine Rolle und Risikostufe, erstelle eine Compliance-Checkliste.

Rollen und relevante Kapitel:
- Anbieter: Kapitel III, Abschnitt 2 (Art. 8-15) für Hochrisiko-Anforderungen, Art. 16 für Anbieterpflichten
- Betreiber: Art. 26 für Betreiberpflichten
- Einführer: Art. 23 für Einführerpflichten
- Händler: Art. 24 für Händlerpflichten

Nutze die Tools, um den tatsächlichen Verordnungstext für jede Pflicht nachzuschlagen.

Antworte abschließend mit einem JSON-Objekt in folgender Struktur:
{
  "role": "provider" | "deployer" | "importer" | "distributor",
  "risk_level": "high" | "limited" | "minimal",
  "obligations": [
    {"obligation": "...", "article": "Art. X", "description": "...", "priority": "critical" | "important" | "recommended"}
  ],
  "summary": "Zusammenfassung der Compliance-Anforderungen",
  "citations": [{"article": "Art. X", "title": "...", "relevance": "..."}]
}

Antworte auf Deutsch."""

## Anbieter eines Hochrisiko-KI-Systems

In [ ]:
response, messages = run_agent(
    client=client,
    system_prompt=ADVISOR_SYSTEM_PROMPT,
    user_message="""Ich bin Anbieter eines Hochrisiko-KI-Systems, das für die Personalrekrutierung 
und Kandidatenvorauswahl eingesetzt wird. Welche Compliance-Pflichten habe ich gemäß der EU-KI-Verordnung?""",
    tools=TOOL_SCHEMAS,
)

print("=== Tool-Aufrufe ===")
for m in messages:
    if isinstance(m, dict) and m.get('tool_calls'):
        for tc in m['tool_calls']:
            print(f"  {tc['function']['name']}({tc['function']['arguments'][:80]})")

print(f"\n=== Antwort ===\n{response[:3000]}")

## Betreiber eines Hochrisiko-KI-Systems

In [ ]:
response2, messages2 = run_agent(
    client=client,
    system_prompt=ADVISOR_SYSTEM_PROMPT,
    user_message="""Ich setze ein Hochrisiko-KI-System zur Kreditwürdigkeitsprüfung in einer Bank ein.
Welche Pflichten habe ich als Betreiber?""",
    tools=TOOL_SCHEMAS,
)

print(f"=== Antwort ===\n{response2[:3000]}")

## Multi-Turn: Folgefrage

In [ ]:
# Gespräch vom Betreiber-Szenario fortsetzen
messages2.append({"role": "user", "content": "Was ist mit der Grundrechte-Folgenabschätzung? Können Sie mir mehr Details zu dieser spezifischen Pflicht geben?"})

followup, messages2 = run_agent(
    client=client,
    system_prompt=ADVISOR_SYSTEM_PROMPT,
    user_message="",  # not used when messages provided
    tools=TOOL_SCHEMAS,
    messages=messages2,
)

print(f"=== Folgeantwort ===\n{followup[:2000]}")